In [2]:
# Exercise 5

In [3]:
import math
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pandas as pd
import random

import distance as dis
import stats
import subgraphs as sg

In [4]:
alz_data = pd.read_csv("datasets/AlzheimersDiseaseTrain.csv")
alz_data.head()

,CLASS,AD,AD.1,AD.2,AD.3,AD.4,AD.5,AD.6,AD.7,AD.8,...,NDC.30,NDC.31,NDC.32,NDC.33,NDC.34,NDC.35,NDC.36,NDC.37,NDC.38,NDC.39
0,ANG_1,4.520104,4.430224,4.244593,2.906250,3.106781,1.966611,3.670303,3.365818,4.404305,...,2.687549,3.918413,3.904447,3.173363,3.819013,2.087533,3.460492,3.070150,3.561487,2.153643
1,BDNF_1,2.692648,0.945860,2.916182,-0.262353,0.084827,-0.024774,0.691213,-0.306191,0.397225,...,1.255353,-0.465786,0.099672,1.109036,1.962476,1.739724,2.046212,1.473774,1.710057,0.868764
2,BLC_1,0.788355,0.243013,1.029638,-0.651429,-0.530144,-0.466503,-0.207471,-0.598578,-0.247810,...,-1.047142,-0.752896,-0.646474,-0.577310,0.163524,-0.063873,1.252525,0.851452,0.189141,-0.184288
3,BMP-4_1,1.142045,0.407405,1.966551,-0.032887,0.239389,-0.309028,1.110751,0.516942,1.267004,...,0.713331,-0.160065,0.647319,-0.091960,1.239154,2.019634,2.652180,2.394194,1.713033,1.956316
4,BMP-6_1,-0.376951,0.480273,2.397770,-0.455636,-0.567477,-0.444746,0.603011,-0.210501,0.447816,...,-0.288518,-0.386555,-0.378309,-0.386273,0.467163,1.440439,1.933714,0.256852,1.135844,1.599819


In [5]:
# Feature (protein) labels
attr_labels = []
for label in alz_data.iloc[:, 0]:
    attr_labels.append(str(label))
    
# Instance labels
inst_labels = []
for label in alz_data.columns:
    inst_labels.append(str(label))
inst_labels.remove('CLASS')
print(inst_labels)

['AD', 'AD.1', 'AD.2', 'AD.3', 'AD.4', 'AD.5', 'AD.6', 'AD.7', 'AD.8', 'AD.9', 'AD.10', 'AD.11', 'AD.12', 'AD.13', 'AD.14', 'AD.15', 'AD.16', 'AD.17', 'AD.18', 'AD.19', 'AD.20', 'AD.21', 'AD.22', 'AD.23', 'AD.24', 'AD.25', 'AD.26', 'AD.27', 'AD.28', 'AD.29', 'AD.30', 'AD.31', 'AD.32', 'AD.33', 'AD.34', 'AD.35', 'AD.36', 'AD.37', 'AD.38', 'AD.39', 'AD.40', 'AD.41', 'AD.42', 'NDC', 'NDC.1', 'NDC.2', 'NDC.3', 'NDC.4', 'NDC.5', 'NDC.6', 'NDC.7', 'NDC.8', 'NDC.9', 'NDC.10', 'NDC.11', 'NDC.12', 'NDC.13', 'NDC.14', 'NDC.15', 'NDC.16', 'NDC.17', 'NDC.18', 'NDC.19', 'NDC.20', 'NDC.21', 'NDC.22', 'NDC.23', 'NDC.24', 'NDC.25', 'NDC.26', 'NDC.27', 'NDC.28', 'NDC.29', 'NDC.30', 'NDC.31', 'NDC.32', 'NDC.33', 'NDC.34', 'NDC.35', 'NDC.36', 'NDC.37', 'NDC.38', 'NDC.39']


In [6]:
alz_data = alz_data.iloc[:, 1:]
alz_data.head()

,AD,AD.1,AD.2,AD.3,AD.4,AD.5,AD.6,AD.7,AD.8,AD.9,...,NDC.30,NDC.31,NDC.32,NDC.33,NDC.34,NDC.35,NDC.36,NDC.37,NDC.38,NDC.39
0,4.520104,4.430224,4.244593,2.906250,3.106781,1.966611,3.670303,3.365818,4.404305,4.521486,...,2.687549,3.918413,3.904447,3.173363,3.819013,2.087533,3.460492,3.070150,3.561487,2.153643
1,2.692648,0.945860,2.916182,-0.262353,0.084827,-0.024774,0.691213,-0.306191,0.397225,0.276861,...,1.255353,-0.465786,0.099672,1.109036,1.962476,1.739724,2.046212,1.473774,1.710057,0.868764
2,0.788355,0.243013,1.029638,-0.651429,-0.530144,-0.466503,-0.207471,-0.598578,-0.247810,-0.569392,...,-1.047142,-0.752896,-0.646474,-0.577310,0.163524,-0.063873,1.252525,0.851452,0.189141,-0.184288
3,1.142045,0.407405,1.966551,-0.032887,0.239389,-0.309028,1.110751,0.516942,1.267004,0.140410,...,0.713331,-0.160065,0.647319,-0.091960,1.239154,2.019634,2.652180,2.394194,1.713033,1.956316
4,-0.376951,0.480273,2.397770,-0.455636,-0.567477,-0.444746,0.603011,-0.210501,0.447816,-0.509188,...,-0.288518,-0.386555,-0.378309,-0.386273,0.467163,1.440439,1.933714,0.256852,1.135844,1.599819


In [7]:
# Attributes are rows, instances are columns
nattr, ninst = alz_data.shape
D = np.zeros((ninst, ninst))

for i in range(ninst):
    i_data = alz_data.iloc[:, i].values
    for j in range(i, ninst):
        j_data = alz_data.iloc[:, j].values
        
        dist = dis.eucl_dist(i_data, j_data)
        D[i][j] = D[j][i] = dist

In [8]:
# Generate sets of vertices
V = set(n for n in range(len(D)))

In [9]:
# Generate the MST and KNN graph then use these to produce the MST-KNN graph
k = 2
mst = sg.mst(V, D, "output/alzheimersMST.gml", inst_labels)
knn = sg.knn(V, D, k, "output/alzheimersKNN.gml", inst_labels)
mst_knn = sg.mst_knn(mst, knn, "output/alzheimersMST-KNN.gml")

In [10]:
# Exercise 6

In [11]:
# Set k to the same amount of clusters in the mst_knn, seen in the generated graph
k = 9

In [12]:
class Cluster:
    def __init__(self, centroid):
        self.centroid = centroid
        self.vertices = [centroid]
        
    def __str__(self):
        return str(self.vertices)
        
    def new_centroid(self):
        distances = []
        for v in self.vertices:
            distance = 0
            for w in self.vertices:
                if v == w:
                    continue
                    
                distance += D[v][w]**2
                
            distances.append(distance)
            
        min_dist = distances[0]
        index = 0
        for i, distance in enumerate(distances):
            if distance < min_dist:
                min_dist = distance
                index = i
                
        self.centroid = self.vertices[index]
    
    def clear_cluster(self):
        self.vertices.clear()
        self.vertices.append(self.centroid)

In [13]:
# Generate k clusters and pick k random instances to be the centroids
centroids = random.sample(list(V), k)  # Samples selects k items from the list with no duplicates
print(centroids)

clusters = []
for centroid in centroids:
    clusters.append(Cluster(centroid))

[34, 2, 72, 67, 11, 81, 77, 38, 40]


In [14]:
loop = True
while loop:
    loop = False
    # Iterate through all instances and add it to the cluster with the nearest centroid
    for inst in V:
        if inst in centroids:
            continue
        min_dist = D[inst][clusters[0].centroid]
        closest_clust = 0
        for i, cluster in enumerate(clusters):    
            dist = D[inst][cluster.centroid]
            if dist < min_dist:
                min_dist = dist
                closest_clust = i

        clusters[closest_clust].vertices.append(inst)

    # Recalculate centroids, if a change is detected continue looping
    centroids.clear()
    for cluster in clusters:
        current_c = cluster.centroid
        cluster.new_centroid()
        
        new_c = cluster.centroid
        centroids.append(new_c)
        
        cluster.clear_cluster()
        if current_c != new_c:
            loop = True

In [15]:
# Iterate through all instances and add it to the cluster with the nearest centroid
for inst in V:
    if inst in centroids:
        continue
    min_dist = D[inst][clusters[0].centroid]
    closest_clust = 0
    for i, cluster in enumerate(clusters):        
        dist = D[inst][cluster.centroid]
        if dist < min_dist:
            min_dist = dist
            closest_clust = i

    clusters[closest_clust].vertices.append(inst)

In [20]:
for cluster in clusters:
    print(cluster)

[29, 3, 9, 13, 14, 18, 23, 30, 31, 34, 36, 37, 46, 48, 49, 50, 51, 53, 56, 64, 68, 69, 74, 76]
[2, 0]
[70, 6, 8, 47, 55, 71, 72, 73, 75]
[67, 1, 19, 52, 54, 66]
[12, 5, 11, 17, 20, 21, 22, 24, 25, 27, 28, 33, 57, 58, 59, 60, 61, 62, 63, 65]
[43, 16, 44, 45, 78, 79, 80, 81, 82]
[77, 26, 32, 35]
[38, 4, 41, 42]
[40, 7, 10, 15, 39]


In [16]:
# Retrieve class labels of k-means clusters for visual comparison with MST-KNN graph
for cluster in clusters:
    class_list = []
    for v in cluster.vertices:
        class_list.append(inst_labels[v])
        
    print(class_list)

['AD.29', 'AD.3', 'AD.9', 'AD.13', 'AD.14', 'AD.18', 'AD.23', 'AD.30', 'AD.31', 'AD.34', 'AD.36', 'AD.37', 'NDC.3', 'NDC.5', 'NDC.6', 'NDC.7', 'NDC.8', 'NDC.10', 'NDC.13', 'NDC.21', 'NDC.25', 'NDC.26', 'NDC.31', 'NDC.33']
['AD.2', 'AD']
['NDC.27', 'AD.6', 'AD.8', 'NDC.4', 'NDC.12', 'NDC.28', 'NDC.29', 'NDC.30', 'NDC.32']
['NDC.24', 'AD.1', 'AD.19', 'NDC.9', 'NDC.11', 'NDC.23']
['AD.12', 'AD.5', 'AD.11', 'AD.17', 'AD.20', 'AD.21', 'AD.22', 'AD.24', 'AD.25', 'AD.27', 'AD.28', 'AD.33', 'NDC.14', 'NDC.15', 'NDC.16', 'NDC.17', 'NDC.18', 'NDC.19', 'NDC.20', 'NDC.22']
['NDC', 'AD.16', 'NDC.1', 'NDC.2', 'NDC.35', 'NDC.36', 'NDC.37', 'NDC.38', 'NDC.39']
['NDC.34', 'AD.26', 'AD.32', 'AD.35']
['AD.38', 'AD.4', 'AD.41', 'AD.42']
['AD.40', 'AD.7', 'AD.10', 'AD.15', 'AD.39']


In [17]:
# Start with a set for each instance
mst_knn_clusters = [{i} for i in range(len(V))]

# Iterate through mst_knn edges to produce a list of clusters in a similar format to the k-means result
for edge in mst_knn:
    source = inst_labels.index(edge[0])
    target = inst_labels.index(edge[1])
    
    s_set = set()
    t_set = set()
    # Find which sets the source and target are in
    for i in range(len(mst_knn_clusters)):
        if source in mst_knn_clusters[i]:
            s_set = mst_knn_clusters[i]
        if target in mst_knn_clusters[i]:
            t_set = mst_knn_clusters[i]
            
    # Merge these two sets
    new_set = s_set.union(t_set)
    mst_knn_clusters.remove(s_set)
    if s_set != t_set:  # Only remove second set if it is different to the first
        mst_knn_clusters.remove(t_set)
    mst_knn_clusters.append(new_set)
    
mst_knn_list = []
for item in mst_knn_clusters:
    mst_knn_list.append(list(item))
    
for item in mst_knn_list:
    print(item)

[0, 2, 50, 51, 52, 53, 54, 56, 60, 61, 62, 63]
[3, 4, 5, 46]
[65, 66, 67, 21, 22, 57, 58, 59]
[17, 76, 25]
[41, 42, 38]
[48, 68, 36, 69, 74, 75]
[1, 7, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 37, 39, 40, 49, 64, 71, 77]
[80, 81, 82, 43, 44, 45, 78, 79]
[70, 55, 72, 73, 6, 8, 47]


In [25]:
# Ground truth, two clusters, one for each class
gt = []
gt.append([i for i in range(43)])
gt.append([i for i in range(43, 83)])
print(gt)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42], [43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82]]


In [26]:
# Build confusion matrix for purpose of inter-rater reliability calculation
cm = np.zeros((2,2))

# pair considered is either (t = together or a = apart) in the set of clusters
#    mst-knn
#     t a
#gt t
#   a

# Iterate through every possible pair of instances/vertices
for u in range(len(V)):
    for v in range(u + 1, len(V)):
        for cluster in mst_knn_list:
            if u in cluster:
                break
                
        if v in cluster:
            j = 0
        else:
            j = 1
            
        for cluster2 in gt:
            if v in cluster2:
                break
                
        if u in cluster2:
            i = 0
        else:
            i = 1
            
        cm[i][j] += 1
        
print(cm)

[[ 502. 1181.]
 [ 164. 1556.]]


In [27]:
kappa = stats.ckappa(cm)
print(kappa)

0.20425560915531965


In [28]:
# Build confusion matrix for purpose of inter-rater reliability calculation
cm2 = np.zeros((2,2))

# pair considered is either (t = together or a = apart) in the set of clusters
#    k-means
#     t a
#gt t
#   a

# Iterate through every possible pair of instances/vertices
for u in range(len(V)):
    for v in range(u + 1, len(V)):
        for cluster in clusters:
            if u in cluster.vertices:
                break
                
        if v in cluster.vertices:
            j = 0
        else:
            j = 1
            
        for cluster2 in gt:
            if v in cluster2:
                break
                
        if u in cluster2:
            i = 0
        else:
            i = 1
            
        cm2[i][j] += 1
        
print(cm2)

[[ 303. 1380.]
 [ 273. 1447.]]


In [29]:
kappa = stats.ckappa(cm2)
print(kappa)

0.021466589638693766


In [21]:
# Exercise 8

In [49]:
# Single linkage hierarchical clustering using an agglomerative approach
# Start with each instance in its own cluster
hclusters = [[i] for i in range(len(V))]

# Find two closest clusters and merge, repeat until there are k clusters
while(len(hclusters) > k):
    # Set initial distance to that of first two distinct clusters
    min_dist = D[hclusters[0][0]][hclusters[1][0]]
    source = hclusters[0]
    target = hclusters[1]

    for hcluster in hclusters:
        # Compare instances in the current cluster to all instances in other clusters to find the closest cluster
        for inst1 in hcluster:
            for inst2 in V:
                if inst2 in hcluster:
                    continue

                if D[inst1][inst2] < min_dist:
                    min_dist = D[inst1][inst2]
                    source = hcluster
                    target_instance = inst2

    # Find which cluster the other instance is in
    for hcluster in hclusters:
        for inst in hcluster:
            if inst == target_instance:
                target = hcluster

    # Merge the two clusters the two closest instances belong to
    new_cluster = source + target
    hclusters.remove(source)
    hclusters.remove(target)
    hclusters.append(new_cluster)
    
for hcluster in hclusters:
    print(hcluster)

[2]
[6]
[8]
[47]
[79]
[80]
[82]
[78, 45, 44, 43, 81]
[0, 18, 19, 7, 10, 49, 64, 69, 23, 41, 38, 42, 73, 48, 72, 55, 70, 76, 17, 25, 1, 67, 53, 54, 61, 56, 62, 50, 51, 52, 60, 63, 46, 5, 3, 4, 66, 71, 26, 32, 77, 36, 59, 24, 75, 68, 74, 11, 28, 34, 35, 14, 29, 30, 9, 13, 37, 31, 20, 33, 12, 27, 57, 58, 65, 21, 22, 39, 40, 15, 16]


In [50]:
# Retrieve class labels of hierarchical clustering
for hcluster in hclusters:
    class_list = []
    for v in hcluster:
        class_list.append(inst_labels[v])
        
    print(class_list)

['AD.2']
['AD.6']
['AD.8']
['NDC.4']
['NDC.36']
['NDC.37']
['NDC.39']
['NDC.35', 'NDC.2', 'NDC.1', 'NDC', 'NDC.38']
['AD', 'AD.18', 'AD.19', 'AD.7', 'AD.10', 'NDC.6', 'NDC.21', 'NDC.26', 'AD.23', 'AD.41', 'AD.38', 'AD.42', 'NDC.30', 'NDC.5', 'NDC.29', 'NDC.12', 'NDC.27', 'NDC.33', 'AD.17', 'AD.25', 'AD.1', 'NDC.24', 'NDC.10', 'NDC.11', 'NDC.18', 'NDC.13', 'NDC.19', 'NDC.7', 'NDC.8', 'NDC.9', 'NDC.17', 'NDC.20', 'NDC.3', 'AD.5', 'AD.3', 'AD.4', 'NDC.23', 'NDC.28', 'AD.26', 'AD.32', 'NDC.34', 'AD.36', 'NDC.16', 'AD.24', 'NDC.32', 'NDC.25', 'NDC.31', 'AD.11', 'AD.28', 'AD.34', 'AD.35', 'AD.14', 'AD.29', 'AD.30', 'AD.9', 'AD.13', 'AD.37', 'AD.31', 'AD.20', 'AD.33', 'AD.12', 'AD.27', 'NDC.14', 'NDC.15', 'NDC.22', 'AD.21', 'AD.22', 'AD.39', 'AD.40', 'AD.15', 'AD.16']


In [52]:
cm3 = np.zeros([2,2])

# Iterate through every possible pair of instances/vertices
for u in range(len(V)):
    for v in range(u + 1, len(V)):
        for cluster in hclusters:
            if u in cluster:
                break
                
        if v in cluster:
            j = 0
        else:
            j = 1
            
        for cluster2 in gt:
            if v in cluster2:
                break
                
        if u in cluster2:
            i = 0
        else:
            i = 1
            
        cm3[i][j] += 1
        
print(cm3)

kappa = stats.ckappa(cm3)
print(kappa)

[[1255.  428.]
 [1240.  480.]]
0.024634147850251305


In [58]:
# Average linkage hierarchical clustering using an agglomerative approach
# Start with each instance in its own cluster
hclusters = [[i] for i in range(len(V))]

# Find two closest clusters and merge, repeat until there are k clusters
while(len(hclusters) > k):
    # Set initial distance to that of first two distinct clusters
    min_dist = 0
    for inst1 in hclusters[0]:
        for inst2 in hclusters[1]:
            min_dist += D[inst1][inst2]

    min_dist = min_dist / (len(hclusters[0]) + len(hclusters[1]))
    source = hclusters[0]
    target = hclusters[1]

    # Find closest two clusters by average linkage
    for hcluster in hclusters:           
        for hcluster2 in hclusters:
            if hcluster == hcluster2:
                continue

            avg_dist = 0
            for inst1 in hcluster:
                for inst2 in hcluster2:
                    avg_dist += D[inst1][inst2]
                    
            avg_dist = avg_dist / (len(hcluster) + len(hcluster2))
            
            if avg_dist < min_dist:
                min_dist = avg_dist
                source = hcluster
                target = hcluster2

    # Merge the two clusters the two closest instances belong to
    new_cluster = source + target
    hclusters.remove(source)
    hclusters.remove(target)
    hclusters.append(new_cluster)
    
for hcluster in hclusters:
    print(hcluster)

[38, 42, 41, 46, 5, 3, 4]
[82, 43, 81, 44, 45, 78, 79]
[76, 17, 25, 50, 53, 51, 52]
[68, 74, 71, 75, 48, 72, 0, 2]
[61, 56, 62, 60, 63, 59, 69, 1, 23]
[39, 40, 15, 16, 47, 6, 8]
[7, 10, 9, 13, 37, 73, 55, 70, 49, 14, 36]
[26, 32, 34, 35, 29, 30, 31, 77, 18, 19, 64, 80]
[24, 20, 28, 12, 27, 11, 33, 57, 66, 54, 67, 58, 65, 21, 22]


In [59]:
# Retrieve class labels of hierarchical clustering
for hcluster in hclusters:
    class_list = []
    for v in hcluster:
        class_list.append(inst_labels[v])
        
    print(class_list)

['AD.38', 'AD.42', 'AD.41', 'NDC.3', 'AD.5', 'AD.3', 'AD.4']
['NDC.39', 'NDC', 'NDC.38', 'NDC.1', 'NDC.2', 'NDC.35', 'NDC.36']
['NDC.33', 'AD.17', 'AD.25', 'NDC.7', 'NDC.10', 'NDC.8', 'NDC.9']
['NDC.25', 'NDC.31', 'NDC.28', 'NDC.32', 'NDC.5', 'NDC.29', 'AD', 'AD.2']
['NDC.18', 'NDC.13', 'NDC.19', 'NDC.17', 'NDC.20', 'NDC.16', 'NDC.26', 'AD.1', 'AD.23']
['AD.39', 'AD.40', 'AD.15', 'AD.16', 'NDC.4', 'AD.6', 'AD.8']
['AD.7', 'AD.10', 'AD.9', 'AD.13', 'AD.37', 'NDC.30', 'NDC.12', 'NDC.27', 'NDC.6', 'AD.14', 'AD.36']
['AD.26', 'AD.32', 'AD.34', 'AD.35', 'AD.29', 'AD.30', 'AD.31', 'NDC.34', 'AD.18', 'AD.19', 'NDC.21', 'NDC.37']
['AD.24', 'AD.20', 'AD.28', 'AD.12', 'AD.27', 'AD.11', 'AD.33', 'NDC.14', 'NDC.23', 'NDC.11', 'NDC.24', 'NDC.15', 'NDC.22', 'AD.21', 'AD.22']


In [60]:
cm4 = np.zeros([2,2])

# Iterate through every possible pair of instances/vertices
for u in range(len(V)):
    for v in range(u + 1, len(V)):
        for cluster in hclusters:
            if u in cluster:
                break
                
        if v in cluster:
            j = 0
        else:
            j = 1
            
        for cluster2 in gt:
            if v in cluster2:
                break
                
        if u in cluster2:
            i = 0
        else:
            i = 1
            
        cm4[i][j] += 1
        
print(cm4)

kappa = stats.ckappa(cm4)
print(kappa)

[[ 217. 1466.]
 [ 157. 1563.]]
0.037975038524934605
